# Entity Extraction with Generative Models

This notebook demonstrates how to use Cohere's generative models to extract the name of a film from the title of an article. This demonstrates Named Entity Recognition (NER) of entities which are harder to isolate using other NLP methods (and where pre-training provides the model with some context on these entities). This also demonstrates the broader usecase of sturctured generation based on providing multiple examples in the prompt.



![Extracting Entities from text](https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/keyword-extraction-gpt-models.png)


We'll use post titles from the r/Movies subreddit. And for each title, we'll extract which movie the post is about. If the model is unable to detect the name of a movie being mentioned, it will return "none".

## Setup
Let's start by installing the packages we need.

!pip install cohere requests tqdm

We'll then import these packages and declare the function that retrieves post titles from reddit.

In [3]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)

def get_post_titles(**kwargs):
    """ Gets data from the pushshift api. Read more: https://github.com/pushshift/api """
    base_url = f"https://api.pushshift.io/reddit/search/submission/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return [a['title'] for a in request.json()['data']]


You'll need your API key for this next cell. [Sign up to Cohere](https://os.cohere.ai/) and get one if you haven't yet.

In [4]:
# Paste your API key here. Remember to not share publicly
api_key = 'io6D71AQgkNeebtp3AsSff8MacqU0rfalhBGL00f'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

## Preparing examples for the prompt

In our prompt, we'll present the model with examples for the type of output we're after. We basically get a set of subreddit article titles, and label them ourselves. The label here is the name of the movie mentioned in the title (and "none" if no movie is mentioned).


![Labeled dataset of text and extracted text](https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/keyword-extraction-dataset.png)



In [ ]:

movie_examples = [
("Deadpool 2", "Deadpool 2 | Official HD Deadpool's \"Wet on Wet\" Teaser | 2018"),
("none", "Jordan Peele Just Became the First Black Writer-Director With a $100M Movie Debut"),
("Joker", "Joker Officially Rated “R”"),
("Free Guy", "Ryan Reynolds’ 'Free Guy' Receives July 3, 2020 Release Date - About a bank teller stuck in his routine that discovers he’s an NPC character in brutal open world game."),
("none", "James Cameron congratulates Kevin Feige and Marvel!"),
("Guardians of the Galaxy", "The Cast of Guardians of the Galaxy release statement on James Gunn"),
]




## Creating the extraction prompt

We'll create a prompt that demonstrates the task to the model. The prompt contains the examples above, and then presents the input text and asks the model to extract the movie name.


![Extraction prompt containing the examples and the input text](https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/extraction-prompt-example.png)


In [9]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_description, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_description = task_description
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_description +
                "\n---\n".join([examples[i] + "\n" +
                               self.example_prompt +
                               labels[i] for i in range(len(examples))]))

    def extract(self, example):
        extraction = cohere.generate(
            model='large',
            prompt=self.make_prompt(example),
            max_tokens=10,
            temperature=0.1,
            stop_sequences=["\n"])
        return extraction.generations[0].text[:-1]

# Placeholder for movie_examples - Replace this with your actual data
movie_examples = [
    ("label1", "text1"),
    ("label2", "text2"),
    # Add more examples as needed
]

cohereMovieExtractor = cohereExtractor([e[1] for e in movie_examples],
                                       [e[0] for e in movie_examples], [],
                                       "",
                                       "extract the movie title from the post:")

print(cohereMovieExtractor.make_prompt('<input text here>'))

text1
extract the movie title from the post:label1
---
text2
extract the movie title from the post:label2
---
<input text here>
extract the movie title from the post:


In [10]:
# This is what the prompt looks like:
print(cohereMovieExtractor.make_prompt('<input text here>'))

text1
extract the movie title from the post:label1
---
text2
extract the movie title from the post:label2
---
<input text here>
extract the movie title from the post:


## Getting the data
Let's now make the API call to get the top posts for 2021 from r/movies.

In [13]:
def get_post_titles(**kwargs):
    """Gets data from the pushshift api. Read more: https://github.com/pushshift/api"""
    base_url = f"https://api.pushshift.io/reddit/search/submission/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    
    # Check if 'data' key exists in the response
    if 'data' in request.json():
        return [a['title'] for a in request.json()['data']]
    else:
        # Handle the case where 'data' key is not present
        print("Error: 'data' key not found in the JSON response.")
        return []

# Define the number of posts to retrieve
num_posts = 10

# Retrieve movie titles from the "movies" subreddit within a specific time range
movies_list = get_post_titles(
    size=num_posts,
    after=str(int(datetime.datetime(2021, 1, 1, 0, 0).timestamp())),
    before=str(int(datetime.datetime(2022, 1, 1, 0, 0).timestamp())),
    subreddit="movies",
    sort_type="score",
    sort="desc"
)

# Show the list of movie titles
movies_list

Error: 'data' key not found in the JSON response.


[]

## Running the model
And now we loop over the posts and process each one of them with our extractor.

In [14]:
results = []
for text in tqdm(movies_list):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

0it [00:00, ?it/s]


Let's look at the results:

In [15]:
pd.DataFrame(data={'text': movies_list, 'extracted_text': results})

,text,extracted_text


Looking at these results, the model got 9/10 correctly. It didn't pick up on Shaolin Soccer and God of Gambler in example \#4. It also called the second example "Pixar's Luca" instead of "Luca". But maybe we'll let this one slide.

When experimenting with extrction prompts, we'll often find edge-cases along the way. What if a post has two movies mentioned, for example? The more we run into such examples, the more examples we can add to the prompt that address these cases.

## How well does this work?
We can better measure the performance of this extraction method using a larger labeled dataset. So let's load a test set of 100 examples:

In [16]:
test_df = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/movie_extraction_test_set_100.csv',index_col=0)
test_df

,text,label
0,Disney's streaming service loses some movies due to old licensing deals,none
1,"Hi, I’m Sam Raimi, producer of THE GRUDGE which hits theaters tonight. Ask Me Anything!",The Grudge
2,'Parasite' Named Best Picture by Australia's AACTA Awards,Parasite
3,Danny Trejo To Star In Vampire Spaghetti Western ‘Death Rider in the House of Vampires’,Death Rider in the House of Vampires
4,I really wish the 'realistic' CGI animal trend would end.,none
...,...,...
95,Hair Love | Oscar Winning Short Film (Full),Hair Love
96,First image of Jason Alexander in Christian film industry satire 'Faith Based',Faith Based
97,"'Borderlands' Movie in the Works From Eli Roth, Lionsgate",Borderlands
98,"Taika Waititi putting his Oscar ""away"" after winning best adapted screenplay for JOJO RABBIT",Jojo Rabbit


Let's run the extractor on these post titles (calling the API in parallel for quicker results):

In [17]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['text']):
        extracted.append(str(i).strip())
# Save results
test_df['extracted_text'] = extracted

AttributeError: module 'cohere' has no attribute 'generate'

Let's look at some results:

In [18]:
test_df.head()

,text,label
0,Disney's streaming service loses some movies due to old licensing deals,none
1,"Hi, I’m Sam Raimi, producer of THE GRUDGE which hits theaters tonight. Ask Me Anything!",The Grudge
2,'Parasite' Named Best Picture by Australia's AACTA Awards,Parasite
3,Danny Trejo To Star In Vampire Spaghetti Western ‘Death Rider in the House of Vampires’,Death Rider in the House of Vampires
4,I really wish the 'realistic' CGI animal trend would end.,none


Let's calculate the accuracy by comparing to the labeled examples

In [20]:
# Compare the label to the extracted text
test_df['correct'] = (test_df['label'].str.lower() == test_df['extracted_text'].str.lower()).astype(int)

# Print the accuracy
print(f'Classification accuracy {test_df["correct"].mean() *100}%')

Index(['text', 'label'], dtype='object')


So it seems this prompt works well on this small test set. It's not guaranteed it will do as well on other sets, however. The prompt can be improved by trying on more data, discovering edge cases, and adding more examples to the prompt.

We can look at the examples it got wrong:

In [21]:
test_df[test_df['correct']==0]

KeyError: 'correct'

It indeed failed to pick up a few examples. Sometimes this uncovers edge cases and understandable mistakes (e.g. two films are mentioned in the text). 


We can look at the classification report for a more detailed look at what's included in the test set, and what the model got right and wrong:

In [23]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

print(classification_report(test_df['label'].str.lower(), test_df['extracted_text'].str.lower()))

KeyError: 'extracted_text'

This type of extraction is interesting because it doesn't just blindly look at the text. The model has picked up on movie information during its pretraining process and that helps it understand the task from only a few examples.

You can think about extending this to other subreddits, to extract other kinds of entities and information. [Let us know in the forum](https://community.cohere.ai/) what you experiment with and what kinds of results you see!

Happy building!